In [19]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch import nn, optim
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable


In [4]:
X = torch.tensor(np.arange(0, 31), dtype=torch.float32).view(-1, 1)
Y = torch.tensor(np.array([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41, 40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53]), dtype=torch.float32).view(-1, 1)


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(1, 32)
        self.layer2 = nn.Linear(32, 64)
        self.layer3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x


In [6]:
def train_and_test_nn(train_indices, test_indices, epochs):
    model = NeuralNetwork()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()

    train_X = X[train_indices]
    train_Y = Y[train_indices]
    test_X = X[test_indices]
    test_Y = Y[test_indices]

    num_epochs = epochs
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(train_X)
        loss = criterion(output, train_Y)
        loss.backward()
        optimizer.step()

    preds_train = model(train_X)
    train_loss = criterion(preds_train, train_Y)

    preds_test = model(test_X)
    test_loss = criterion(preds_test, test_Y)
    return train_loss, test_loss


In [12]:


print("Case (ii): First 20 points as training data")
train_error, test_error = train_and_test_nn(np.arange(20), np.arange(20, 30), 1000)
print(f"\nTrain Error: {train_error}\nTest Error: {test_error}")

print("\nCase (iii): First 10 and last 10 points as training data")
train_error2, test_error2 = train_and_test_nn(np.hstack([np.arange(10), np.arange(21, 31)]), np.arange(10, 21), 1000)
print(f"\nTrain Error: {train_error2}\nTest Error: {test_error2}")

Case (ii): First 20 points as training data

Train Error: 4.439381122589111
Test Error: 9.366402626037598

Case (iii): First 10 and last 10 points as training data

Train Error: 3.2382946014404297
Test Error: 7.333491802215576


In [21]:
#HW 1 Scores:
#1
# Error for Line (Train): 2.242749386808538
# Error for Parabola (Train): 2.1255393482773766
# Error for Polynomial (Train): 0.028351503968806435
# Error for Line (Test): 3.36363873604787
# Error for Parabola (Test): 8.713651781874919
# Error for Polynomial (Test): 28617752784.428474

#2
# Error for Line (Train): 1.851669904329375
# Error for Parabola (Train): 1.8508364115957907
# Error for Polynomial (Train): 0.1638133765080727
# Error for Line (Test): 2.8065076975181618
# Error for Parabola (Test): 2.774982896893291
# Error for Polynomial (Test): 483.9099124568562

In [8]:
import torch
import torchvision
from torchvision import transforms
from sklearn.decomposition import PCA
import numpy as np

# Load the MNIST train and test datasets
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = torchvision.datasets.MNIST('~/.torch/datasets', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST('~/.torch/datasets', train=False, download=True, transform=transform)

# Compute the first 20 PCA modes of the digit images
train_data = train_dataset.data.numpy().reshape(-1, 28*28)
pca = PCA(n_components=20)
pca.fit(train_data)
print(f'First 20 PCA Modes:\n{pca.components_}')

# Loaders for the train and test datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000)

# Define the neural network
class FeedForwardNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(28 * 28, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = FeedForwardNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train the neural network
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Reshape the data and perform the forward pass
        data = data.view(data.size(0), -1)
        predictions = model(data)

        # Compute the loss, gradients, and update the parameters
        loss = criterion(predictions, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for the current epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the neural network
correct = 0
total = 0
with torch.no_grad():
    for data, targets in test_loader:
        # Reshape the data and perform the forward pass
        data = data.view(data.size(0), -1)
        predictions = model(data)

        # Compute the number of correct predictions
        _, predicted = torch.max(predictions.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}')


First 20 PCA Modes:
[[ 6.03863025e-18 -2.06760528e-18 -1.97133939e-19 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 9.83706835e-17 -1.17227770e-17 -9.03127920e-18 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [ 4.07530131e-17 -3.40610710e-17  6.00320808e-18 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 4.77907602e-17  8.45964747e-18  1.45399137e-17 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.93112540e-17  5.12008009e-17 -9.54434652e-17 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 5.45389222e-18 -5.40219756e-17  6.81901851e-17 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Epoch [1/10], Loss: 0.2357
Epoch [2/10], Loss: 0.1875
Epoch [3/10], Loss: 0.1504
Epoch [4/10], Loss: 0.1409
Epoch [5/10], Loss: 0.2250
Epoch [6/10], Loss: 0.0334
Epoch [7/10], Loss: 0.1476
Epoch [8/10], Loss: 0.0308
Epoch [9/10], Loss: 0.0669
Epoch [10/10], Loss: 0.0460
Accuracy of the network on the 10000 test images: 96.78


In [ ]:
#HW3 Scores:
#Tree: 0.8769
#SVM: 0.9792
#LDA: 0.8730


In [14]:
#SVM LDA and Decision Trees
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from keras.datasets import mnist
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [15]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784).T
X_test = X_test.reshape(10000, 784).T

In [16]:
tree = DecisionTreeClassifier(random_state=44)
tree.fit(X_train.T, y_train)
y_pred_tree = tree.predict(X_test.T)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
print(accuracy_tree)

0.8769


In [17]:
svm = SVC(probability=False)
svm.fit(X_train.T, y_train)
y_pred_svm = svm.predict(X_test.T)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(accuracy_svm)

0.9792


In [18]:
lda = LDA()
lda.fit(X_train.T, y_train)
y_pred_lda = lda.predict(X_test.T)
accuracy_lda = accuracy_score(y_test, y_pred_lda)
print(accuracy_lda)

0.873
